**Help taken from :**

[encrypt and decrypt functions](https://gist.github.com/JonCooperWorks/5314103)  
[for the prime generator function](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/)    

In [1]:
import math
import random
import time

In [2]:
# Euclids algorithm to find gcd
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

[Understanding Euclidean Algorithm](https://math.stackexchange.com/questions/747342/extended-euclidean-algorithm-for-modular-inverse)

In [11]:
# extended euclidean algorithm for calculating modular inverse
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, x, y = egcd(b % a, a)
        return (g, y- (b // a) * x, x)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        # this means a and m are not co prime and hence this algorithm cant work
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [4]:
# check whether the number is prime or not
def is_prime(number):
    if number > 1:
        if number == 2:
            return True
        if number % 2 == 0:
            return False
        for current in range(3, int(math.sqrt(number) + 1), 2):
            if number % current == 0:
                return False
        return True
    return False

In [5]:
# get a prime number just greater than a given number
def get_primes(number):
    while True:
        if is_prime(number):
            number = yield number
        number += 1

In [6]:
# generate the prime number for the keys
def generate_primes():    
    
    # to obtain random seeds 
    random.seed(time.time())    
    
    # generate a random number 
    startforp = random.randint(100,200)
    # get the generator function
    prime_handle = get_primes(startforp)
    
    # get the first prime number
    p = prime_handle.send(None)

    # put a respectable gap between the two 
    startforq = startforp + 100
    
    # get the second prime number
    q = prime_handle.send(startforq)
        
    return [p,q]

In [7]:
# generate the public and private keys
def generate_keys():
    
    # obtain two large primes
    primes = generate_primes()
    
    p = primes[0]
    q = primes[1]
    
    # obtain n
    n = p*q
    
    # obtain the euler totient of p and q
    phi = (p-1)*(q-1)
    
    # choosing standard value of e 
    e = 65537
    
    # Use Extended Euclid's Algorithm to generate the private key
    d = modinv(e,phi)
    
    # Return public and private keypair
    # Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))
    

In [8]:
def encrypt(pk, plaintext):
    # Unpack the key into it's components
    key, n = pk
    # Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [(ord(char) ** key) % n for char in plaintext]
    # Return the array of bytes
    return cipher

In [9]:
def decrypt(pk, ciphertext):
    # Unpack the key into its components
    key, n = pk
    # Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr((char ** key) % n) for char in ciphertext]
    # Return the array of bytes as a string
    return ''.join(plain)

In [12]:
if __name__ == '__main__':
    
    public, private = generate_keys()
    print ("Your public key is ", public ," and your private key is ", private)
    message = input("Enter a message to encrypt with your private key: ")
    encrypted_msg = encrypt(private, message)
    
    print ("Your encrypted message is: ")
    print (''.join(map(lambda x: str(x), encrypted_msg)))
    
    print("-------------------------------")
    
    print ("Decrypting message with public key ", public ," . . .")
    print ("Your message is:")
    print (decrypt(public, encrypted_msg))
    

Your public key is  (65537, 46883)  and your private key is  (39113, 46883)
Enter a message to encrypt with your private key: arjun
Your encrypted message is: 
22535867778362801725327
-------------------------------
Decrypting message with public key  (65537, 46883)  . . .
Your message is:
arjun
